**ADVANCE DATA CLEANING**

Creating similarity detection function using FuzzyWuzzy that will show closley related word rows(e.g. "sao joao do pau dalho" vs "sao joao do pau d'alho") and their matching % in order to further rectify them

In [ ]:
#SIMILARITY DETECTION SCRIPT

import pandas as pd
from fuzzywuzzy import fuzz
from sqlalchemy import create_engine

# 1. CONNECT TO POSTGRESQL
username = 'postgres'
password = '**********'  
host = 'localhost'
port = '5432'
database = 'e-commerce project'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
conn = engine.connect()

# 2. LOAD CLEANED COLUMN
df = pd.read_sql("SELECT DISTINCT geolocation_city FROM olist_geolocation", conn)

# 3. PREPARE FOR FUZZY MATCHING
df['cleaned'] = df['geolocation_city'].str.strip().str.lower()
unique_cities = df['cleaned'].dropna().unique()

# 4. FIND SIMILARITY > 90
from itertools import combinations
similar_pairs = []

for city1, city2 in combinations(unique_cities, 2):
    score = fuzz.ratio(city1, city2)
    if 90 < score < 100:  # Similar but not same
        similar_pairs.append((city1, city2, score))

# 5. SHOW RESULTS
similar_df = pd.DataFrame(similar_pairs, columns=['City1', 'City2', 'Similarity'])
similar_df = similar_df.sort_values(by='Similarity', ascending=False)

print(f"🔍 Total possible typo/conflict pairs remaining: {len(similar_df)}")
print(similar_df.head(20))  # View top suspicious ones


🔍 Total possible typo/conflict pairs remaining: 1489
                                 City1                            City2  \
287               campos dos goytacaze            campos dos goytacazes   
526             jaboatão dos gurarapes          jaboatão dos guararapes   
1066            olho d'água das cunhãs            olho dágua das cunhãs   
440           nova brasilandia d oeste          nova brasilandia doeste   
1344            sao joao do pau d'alho            sao joao do pau dalho   
533               santa bárbara doeste            santa bárbara d'oeste   
696           nova brasilandia d'oeste          nova brasilandia doeste   
345               figueiropolis doeste            figueiropolis d'oeste   
1220             figueirópolis d'oeste             figueirópolis doeste   
342               figueiropolis doeste            figueiropolis d oeste   
727          presidente castelo branco       presidente castello branco   
550            barra de  santo antônio         

Creating function that automatically standardizes city names by grouping entries with over 95% similarity and replacing them with most frequent varient

In [ ]:
#Geolocation City Correction Script


import pandas as pd
from fuzzywuzzy import fuzz
from sqlalchemy import create_engine

# Step 1: Connect to PostgreSQL
username = 'postgres'
password = '***********' 
host = 'localhost'
port = '5432'
database = 'e-commerce project'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
conn = engine.connect()

# Step 2: Load geolocation data
df = pd.read_sql("SELECT geo_id, geolocation_city FROM olist_geolocation", conn)
df['geolocation_city_clean'] = df['geolocation_city'].str.lower().str.strip()

# Step 3: Get frequency of each city
city_freq = df['geolocation_city_clean'].value_counts().reset_index()
city_freq.columns = ['city', 'count']

# Step 4: Identify similar cities
from itertools import combinations
similar_map = {}

city_list = city_freq['city'].tolist()

for city1, city2 in combinations(city_list, 2):
    score = fuzz.ratio(city1, city2)
    if 95 < score < 100:
        # Choose the more frequent one as the correct name
        count1 = city_freq[city_freq['city'] == city1]['count'].values[0]
        count2 = city_freq[city_freq['city'] == city2]['count'].values[0]
        correct = city1 if count1 >= count2 else city2
        wrong = city2 if count1 >= count2 else city1
        similar_map[wrong] = correct

# Step 5: Replace wrong values with correct ones
df['geolocation_city_corrected'] = df['geolocation_city_clean'].replace(similar_map)

# Step 6: Update PostgreSQL using geo_id
from sqlalchemy.sql import text

with engine.begin() as connection:
    for index, row in df.iterrows():
        connection.execute(
            text("""
                UPDATE olist_geolocation
                SET geolocation_city = :new_val
                WHERE geo_id = :geo_id
            """),
            {'new_val': row['geolocation_city_corrected'], 'geo_id': row['geo_id']}
        )

print("✅ Successfully corrected similar city names in geolocation_city.")


✅ Successfully corrected similar city names in geolocation_city.


Checking If all the values above 95% are rectified

In [ ]:
#SIMILARITY DETECTION SCRIPT

import pandas as pd
from fuzzywuzzy import fuzz
from sqlalchemy import create_engine


# 1. CONNECT TO POSTGRESQL
username = 'postgres'
password = '**********'
host = 'localhost'
port = '5432'
database = 'e-commerce project'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
conn = engine.connect()

# 2. LOAD CLEANED COLUMN
df = pd.read_sql("SELECT DISTINCT geolocation_city FROM olist_geolocation", conn)

# 3. PREPARE FOR FUZZY MATCHING
df['cleaned'] = df['geolocation_city'].str.strip().str.lower()
unique_cities = df['cleaned'].dropna().unique()

# 4. FIND SIMILARITY > 90
from itertools import combinations
similar_pairs = []

for city1, city2 in combinations(unique_cities, 2):
    score = fuzz.ratio(city1, city2)
    if 90 < score < 100:  # Similar but not same
        similar_pairs.append((city1, city2, score))

# 5. SHOW RESULTS
similar_df = pd.DataFrame(similar_pairs, columns=['City1', 'City2', 'Similarity'])
similar_df = similar_df.sort_values(by='Similarity', ascending=False)

print(f"🔍 Total possible typo/conflict pairs remaining: {len(similar_df)}")
print(similar_df.head(20))  # View top suspicious ones


🔍 Total possible typo/conflict pairs remaining: 358
           City1        City2  Similarity
179   itambaraca    itamaraca          95
138   catanduvas    catanduva          95
185  petrolandia   perolandia          95
184   fronteiras    fronteira          95
164   montenegro  monte negro          95
162    alexandra   alexandria          95
159   charqueada  charqueadas          95
149    pau darco   pau d arco          95
145  itainopolis   itaiopolis          95
134   cearamirim  ceara mirim          95
38    crisopolis  cristopolis          95
126  materlandia   matelandia          95
125   palmeirais    palmeiras          95
100   gameleiras    gameleira          95
97   jaboticabal   jaboticaba          95
94   rio bracnco   rio branco          95
84    lagoa nova  alagoa nova          95
78     alagoinha   alagoinhas          95
193   mogi guacu    mogiguacu          95
205  paragominas   aragominas          95


All the similar rows of similarity above 95% are sucessfully rectified and replaced by most frequent varient which is likely to be correct. To stay safe from mixing up closley similar Brazalian city names as brazil is known for almost identical yet different city names, we only fix those that are over 95% alike by keeping the most common one

CREATING FUNCTION FOR DETECTING ALL ROWS HAVING REPEATING PHRASES 

In [ ]:
#REPEAT PHRASE DETECTION SCRIPT

import pandas as pd
import re
from sqlalchemy import create_engine

# Database connection info
db_name = "e-commerce project"
username = "postgres"
password = "**********"
host = "localhost"
port = "5432"

# Connect to PostgreSQL
engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{db_name}")
conn = engine.connect()

# Read geolocation_city column from table
df = pd.read_sql("SELECT geo_id, geolocation_city FROM olist_geolocation", conn)

# Function to detect repeated word sequences
def has_repeated_phrase(city):
    if pd.isnull(city):
        return False
    words = city.lower().split()
    for i in range(len(words) - 2):  # Check for 2 or 3 word phrases
        phrase = ' '.join(words[i:i+2])
        if phrase and city.lower().count(phrase) > 1:
            return True
    return False

# Filter rows with suspicious repeats
df['has_repeat'] = df['geolocation_city'].apply(has_repeated_phrase)
repeated_df = df[df['has_repeat']]

# Display suspicious rows
print("🔍 Suspicious rows with repeated phrases:\n")
print(repeated_df[['geo_id', 'geolocation_city']].to_string(index=False))

# Optional: Save to CSV for manual review
# repeated_df.to_csv("repeated_cities_check.csv", index=False)


🔍 Suspicious rows with repeated phrases:

 geo_id                     geolocation_city
 998244 rio de janeiro rio de janeiro brasil


Now repeating process and using same similarity detection function for Olist Seller table as some inconsistencies also found in seller city column

In [ ]:
#SIMILARITY DETECTION SCRIPT

import pandas as pd
from fuzzywuzzy import fuzz
from sqlalchemy import create_engine

# 1. CONNECT TO POSTGRESQL
username = 'postgres'
password = '**********'
host = 'localhost'
port = '5432'
database = 'e-commerce project'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
conn = engine.connect()

# 2. LOAD CLEANED COLUMN
df = pd.read_sql("SELECT DISTINCT seller_city FROM olist_sellers", conn)

# 3. PREPARE FOR FUZZY MATCHING
df['cleaned'] = df['seller_city'].str.strip().str.lower()
unique_cities = df['cleaned'].dropna().unique()

# 4. FIND SIMILARITY > 90
from itertools import combinations
similar_pairs = []

for city1, city2 in combinations(unique_cities, 2):
    score = fuzz.ratio(city1, city2)
    if 90 < score < 100:  # Similar but not same
        similar_pairs.append((city1, city2, score))

# 5. SHOW RESULTS
similar_df = pd.DataFrame(similar_pairs, columns=['City1', 'City2', 'Similarity'])
similar_df = similar_df.sort_values(by='Similarity', ascending=False)

print(f"🔍 Total possible typo/conflict pairs remaining: {len(similar_df)}")
print(similar_df.head(20))  # View top suspicious ones


🔍 Total possible typo/conflict pairs remaining: 38
                    City1                   City2  Similarity
0   sao jose do rio pardo  scao jose do rio pardo          98
2   sao  jose dos pinhais    sao jose dos pinhais          98
21   sao jose do rio pret   sao jose do rio preto          98
8   sao bernardo do campo    sao bernardo do capo          98
9   sao bernardo do campo    ao bernardo do campo          98
10  ferraz de vasconcelos  ferraz de  vasconcelos          98
36    sao jose dos pinhas    sao jose dos pinhais          97
17        taboao da serra          tabao da serra          97
11         porto ferreira           portoferreira          96
23           floranopolis           florianopolis          96
18          riberao preto          ribeirao preto          96
19          belo horizont          belo horizonte          96
25             sao  paulo               sao paulo          95
24  santa barbara d´oeste   santa barbara d oeste          95
32   sao bernardo d

Rectifying the rows around 95% - 99% similarity

In [ ]:
#Seller City Correction Script


import pandas as pd
from fuzzywuzzy import fuzz
from sqlalchemy import create_engine

# Step 1: Connect to PostgreSQL
username = 'postgres'
password = '**********'
host = 'localhost'
port = '5432'
database = 'e-commerce project'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database}')
conn = engine.connect()

# Step 2: Load geolocation data
df = pd.read_sql("SELECT seller_id, seller_city FROM olist_sellers", conn)
df['seller_city_clean'] = df['seller_city'].str.lower().str.strip()

# Step 3: Get frequency of each city
city_freq = df['seller_city_clean'].value_counts().reset_index()
city_freq.columns = ['city', 'count']

# Step 4: Identify similar cities
from itertools import combinations
similar_map = {}

city_list = city_freq['city'].tolist()

for city1, city2 in combinations(city_list, 2):
    score = fuzz.ratio(city1, city2)
    if 94 < score < 100:
        # Choose the more frequent one as the correct name
        count1 = city_freq[city_freq['city'] == city1]['count'].values[0]
        count2 = city_freq[city_freq['city'] == city2]['count'].values[0]
        correct = city1 if count1 >= count2 else city2
        wrong = city2 if count1 >= count2 else city1
        similar_map[wrong] = correct

# Step 5: Replace wrong values with correct ones
df['seller_city_corrected'] = df['seller_city_clean'].replace(similar_map)

# Step 6: Update PostgreSQL using geo_id
from sqlalchemy.sql import text

with engine.begin() as connection:
    for index, row in df.iterrows():
        connection.execute(
            text("""
                UPDATE olist_sellers
                SET seller_city = :new_val
                WHERE seller_id = :seller_id
            """),
            {'new_val': row['seller_city_corrected'], 'seller_id': row['seller_id']}
        )

print("✅ Successfully corrected similar city names in geolocation_city.")


✅ Successfully corrected similar city names in geolocation_city.
